In [8]:
import os
os.environ["RUN_MODE"] = "0"

In [9]:
from DataLoader import *
from Data import Data
from Evalute import evaluate_model
from Models.SimpleCF import SimpleCF

In [10]:
from time import time
# An example for running the model and evaluating using leave-1-out and top-k using hit ratio and NCDG metrics
convert_binary = True
load_model = False
testset_percentage = 0.2


print('Started...')

df = get_movielens100k(convert_binary=False)

data = Data(df, seed=42)
t0 = time()
training_set, test_set, n_users, n_movies = data.pre_processing(test_percent=0.5, train_precent=1)

Started...
n_users: 943 n_movies: 1682
get_train_instances done


In [ ]:
# WORK ON EMBEDDINGS

In [11]:
epochs = 10

low_rank_cf_model = SimpleCF()

best_hr=0
best_ndcg = 0
low_rank_cf_model.set_model(n_users, n_movies, n_latent_factors=64)
t0 = time()
mean_hr, mean_ndcg = evaluate_model(low_rank_cf_model, test_set)
print('Init: HR = %.4f, NDCG = %.4f, Eval:[%.1f s]'
      % (mean_hr, mean_ndcg, time()-t0 ))
for epoch in range(epochs):
    t1 = time()
    loss = low_rank_cf_model.fit_once(training_set, batch_size=128, verbose=0)
    t2 = time()
    mean_hr, mean_ndcg = evaluate_model(low_rank_cf_model, test_set)
    t3 = time()
    print('Iteration: %d Fit:[%.1f s]: HR = %.4f, NDCG = %.4f, loss = %.4f, Eval:[%.1f s]'
          % (epoch+1, t2 - t1, mean_hr, mean_ndcg, loss, t3 - t2))
    if mean_hr > best_hr and mean_ndcg > best_ndcg:
        best_hr = mean_hr
        best_ndcg = mean_ndcg
        low_rank_cf_model.save_model()
print('Total time: [%.1f s]' % (time()- t0))

model has been set
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
movie_input (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 64)        60416       user_input[0][0]                 
__________________________________________________________________________________________________
movie_embedding (Embedding)     (None, 1, 64)        107712      movie_input[0][0]                
_________________________________________________________________________